In [16]:
import json

file_path = "geometry.json"

with open(file_path, 'r') as file:
    data = json.load(file)
    
print(data)
features = data["features"]

{'type': 'geoFencingGeometry', 'features': [{'properties': {'id': 'box1', 'name': 'Zone A'}, 'geometry': [[-122.42, 37.78], [-122.41, 37.78], [-122.41, 37.77], [-122.42, 37.77], [-122.42, 37.78]]}, {'properties': {'id': 'box1', 'name': 'Extra Zone A'}, 'geometry': [[-122.42, 37.78], [-122.41, 37.78], [-122.41, 37.77], [-122.42, 37.77], [-122.42, 37.78]]}, {'properties': {'id': 'bigbox', 'name': 'BIG ZONE'}, 'geometry': [[-125.0, 49.3843], [-66.9346, 49.3843], [-66.9346, 24.3963], [-125.0, 24.3963], [-125.0, 49.3843]]}, {'properties': {'id': 'box2', 'name': 'Zone B'}, 'geometry': [[-122.4, 37.79], [-122.39, 37.79], [-122.39, 37.78], [-122.4, 37.78], [-122.4, 37.79]]}, {'properties': {'id': 'box3', 'name': 'Zone C'}, 'geometry': [[-122.43, 37.76], [-122.42, 37.76], [-122.42, 37.75], [-122.43, 37.75], [-122.43, 37.76]]}, {'properties': {'id': 'box4', 'name': 'Zone D'}, 'geometry': [[-122.38, 37.8], [-122.37, 37.8], [-122.37, 37.79], [-122.38, 37.79], [-122.38, 37.8]]}]}


In [17]:
def is_point_inside_polygon(x, y, polygon):
    """
    Determines if a point (x, y) is inside a given polygon using the Ray-Casting method.
    """
    n = len(polygon)
    inside = False

    p1x, p1y = polygon[0]
    for i in range(n + 1):
        p2x, p2y = polygon[i % n]
        if min(p1y, p2y) < y <= max(p1y, p2y) and x <= max(p1x, p2x):
            if p1y != p2y:
                xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
            if p1x == p2x or x <= xinters:
                inside = not inside
        p1x, p1y = p2x, p2y

    return inside

In [18]:
point_x, point_y = -122.415, 37.775
is_inside = is_point_inside_polygon(point_x, point_y, features[0]["geometry"])

print(f"Is the point inside Zone A? {is_inside}")

Is the point inside Zone A? True


In [19]:
def getFences(x, y, geometry):
    box_list = []
    for i in geometry:
        if is_point_inside_polygon(x, y, i["geometry"]):
            box_list.append(i["properties"]["name"])
            
    return box_list

In [20]:
point_x, point_y = -122.415, 37.775
getFences(point_x, point_y, features)

['Zone A', 'Extra Zone A', 'BIG ZONE']

suppose we have a big dataframe
we have set of fences
isolate every consequtive sequence for every zone. - every sequence of rows that coorispond to a time that the car is in a zone
if you have 5 zone. 10 laps. you should have 50 subsets

If we ask we want the times we are in zone 

I ask for zoneA. Give rows that are in zone A

In [21]:
import pandas as pd


drivefile = "0_130000_ciaran_donuts.csv"

df = pd.read_csv(drivefile)

In [ ]:
import pandas as pd

zone_dict = {}

# Check if the required columns exist
if "Latitude|Degrees|-180.0|180.0|25" not in df.columns or "Longitude|Degrees|-180.0|180.0|25" not in df.columns:
    raise ValueError("Latitude or Longitude column missing in DataFrame.")

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    y = row["Latitude|Degrees|-180.0|180.0|25"]
    x = row["Longitude|Degrees|-180.0|180.0|25"]
    
    # Skip rows with missing coordinates
    if pd.isna(x) or pd.isna(y):
        continue
    
    # Get the areas based on the coordinates
    areas = getFences(x, y, features)
    
    # Process each area that contains the point
    for area in areas:
        
        # Initialize the DataFrame for this zone if it doesn't exist
        if area not in zone_dict:
            zone_dict[area] = []
        
        # Append the row as a dictionary to the corresponding zone
        zone_dict[area].append(row.to_dict())

# Convert the lists in zone_dict to DataFrames
for zone, rows in zone_dict.items():
    # Create a DataFrame for each zone
    zone_dict[zone] = pd.DataFrame(rows)

# Now `zone_dict` contains DataFrames where each zone is a key
# Each DataFrame contains the rows for that zone, with columns being the same
# Here is how to print the data in CSV-like format:

for zone, df_zone in zone_dict.items():
    print(f"Zone: {zone}")
    print(df_zone.to_csv(index=False))  # Print the DataFrame as CSV (without index)
    print("\n")


Zone: BIG ZONE
,Unnamed: 0,"Interval|""ms""|0|0|1","Utc|""ms""|0|0|1","Analog4|""Volts""|0.0|5.0|500","Analog5|""Volts""|0.0|5.0|500","Battery|""Volts""|0.0|20.0|1","AccelX|""G""|-3.0|3.0|200","AccelY|""G""|-3.0|3.0|200","AccelZ|""G""|-3.0|3.0|200","Yaw|""Deg/Sec""|-120|120|200","Pitch|""Deg/Sec""|-120|120|200","Roll|""Deg/Sec""|-120|120|200","Gsum|""G""|0.0|3.0|200","RPM|""rpm""|0|8000|500","RPM2|""rpm""|0|8000|500","GPIO1|""""|0.0|1.0|50",Latitude|Degrees|-180.0|180.0|25,Longitude|Degrees|-180.0|180.0|25,Speed|mph|0.0|150.0|25,"Altitude|""ft""|0.0|4000.0|25","GPSSats|""""|0|20|25","GPSQual|""""|0|5|25","GPSDOP|""""|0.0|20.0|25"
0,2853.0,5923.0,1714855088077.0,1.5,1.93,,-0.03,0.02,1.02,0.0,-3.0,-1.0,0.03,0.0,112.0,0.0,39.674324,-77.417816,0.0,1191.6,6.0,2.0,2.0
1,2873.0,5963.0,1714855088115.0,1.46,1.89,,-0.04,0.01,0.9,0.0,-1.0,0.0,0.04,0.0,112.0,0.0,39.674324,-77.417816,0.0,1188.8,6.0,2.0,2.0
2,2893.0,6003.0,1714855088155.0,1.52,1.94,,-0.04,0.02,0.91,0.0,2.0,0.0,0.04,0.0,112.0,0.0,39.